## Running Experiment on Jupyter Notebook

In [1]:
import tensorflow as tf 
from keras import backend as K
from glob import glob
import pydicom
from keras import backend as K
from keras.optimizers import SGD
from keras.models import load_model, Model
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
import cv2
from scipy.ndimage import zoom
from sklearn import preprocessing
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import mlflow

In [2]:
def get_data_for_3d_volumes(data, path, number_idx):
    
    data_to_merge = data[["patient_id", "any_injury"]]
    shuffled_data = data_to_merge.sample(frac=1, random_state=42)
    shuffled_indexes = shuffled_data.index[:number_idx]
    selected_rows = shuffled_data.loc[shuffled_indexes]
    data_to_merge_processed = selected_rows.reset_index()
    
    total_paths = []
    patient_ids = []
    category = []
    
    for patient_id in range(len(data_to_merge_processed)):
    
        p_id = str(data_to_merge_processed["patient_id"][patient_id])
        str_imgs_path = path + p_id + '/'
        patient_img_paths = []

        for file in glob(str_imgs_path + '*'):
            for image_path in glob(file + '/*'):
                 patient_img_paths.append(image_path)
    
        total_paths.append(patient_img_paths)
        patient_ids.append(data_to_merge_processed["patient_id"][patient_id])
        category.append(data_to_merge_processed["any_injury"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, total_paths, category)),
               columns =["Patient_id", "Patient_paths", "Patient_category"])
    
    return final_data 

def training_plot(metrics, history):
    f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 4))
    for idx, metric in enumerate(metrics):
        ax[idx].plot(history.history[metric], ls='dashed')
        ax[idx].set_xlabel("Epochs")
        ax[idx].set_ylabel(metric)
        ax[idx].plot(history.history['val_' + metric]);
        ax[idx].legend([metric, 'val_' + metric])

In [3]:
class Image3DGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size, target_depth=64, target_size=(128,128)):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.target_size = target_size
        self.target_depth = target_depth

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    
    def resize_img(self, img_paths):
        preprocessed_images = []
        for image_path in img_paths: 
            image = pydicom.read_file(image_path)
            image = image.pixel_array
            image = cv2.resize(image, self.target_size)
            image_array = np.array(image)
            preprocessed_images.append(image_array)

    # Create an empty volume array
        volume_shape = (self.target_size[0], self.target_size[1], len(preprocessed_images)) 
        volume = np.zeros(volume_shape, dtype=np.uint16)
    # Populate the volume with images
        for i, image_array in enumerate(preprocessed_images):
            volume[:,:,i] = image_array
        return volume
    
    def change_depth_siz(self, patient_volume):
        desired_depth = self.target_depth
        current_depth = patient_volume.shape[-1]
        depth = current_depth / desired_depth
        depth_factor = 1 / depth
        img_new = zoom(patient_volume, (1, 1, depth_factor), mode='nearest')
        return img_new
    
    def normalize_volume(self, resized_volume):
        original_shape = resized_volume.shape
        flattened_image = resized_volume.reshape((-1,))
        scaler = preprocessing.MinMaxScaler()
        normalized_flattened_image = scaler.fit_transform(flattened_image.reshape((-1, 1)))
        normalized_volume_image = normalized_flattened_image.reshape(original_shape)
        return normalized_volume_image
    
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        resized_images = []
        for list_files in batch_x:
            preprocessed_images = self.resize_img(list_files)
            resized_images_siz = self.change_depth_siz(preprocessed_images)
            normalized_volume = self.normalize_volume(resized_images_siz)
            resized_images.append(normalized_volume)

        resized_images = np.array(resized_images)
        return resized_images, np.array(batch_y)

In [4]:
def convolutional_block_3d(inputs, num_filters):

    x = Conv3D(filters=num_filters, kernel_size=(3,3,3),
    activation="relu")(inputs)
    x = MaxPool3D(pool_size=(2,2,2))(x)
    x = BatchNormalization()(x)

    return x

# function to define the dense block of the network, composed by:
# 2 dense layer with 2 dropout layes in between and one output layer for clasification
def dense_block(flatten_layer):
    dense_layer_1 = Dense(units=512, activation='relu')(flatten_layer)
    dense_layer_1 = Dropout(0.4)(dense_layer_1)

    #dense_layer_2 = Dense(units=256, activation='relu')(dense_layer_1)
    #dense_layer_2 = Dropout(0.4)(dense_layer_2)
    output_layer = Dense(units=2, activation='softmax')(dense_layer_1)

    return output_layer

# Main function to build the 3D Conv Network
def build_3d_network(input_shape):

    input_layer = Input(input_shape)

    x1 = convolutional_block_3d(input_layer, 64)
    x2 = convolutional_block_3d(x1, 64)
    x3 = convolutional_block_3d(x2, 128)
    x4 = convolutional_block_3d(x3, 256)

    flatten_layer = Flatten()(x4)

    output = dense_block(flatten_layer)
    model = Model(inputs=input_layer, outputs=output)

    model.compile(loss='mae',optimizer=SGD(learning_rate=1e-06, momentum=0.99, decay=0.0, nesterov=False), metrics=['acc'])
    
    return model

In [5]:
checkpoint_filepath = 'Experiments_ckpt/experiment__checkpoint.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                                    save_weights_only=True,
                                                                    monitor='val_accuracy',
                                                                    mode='max',
                                                                    save_best_only=True)

train_data = pd.read_csv(f"D:/Downloads/rsna-2023-abdominal-trauma-detection/train.csv")
path = 'D:/Downloads/rsna-2023-abdominal-trauma-detection/train_images/'
paths = get_data_for_3d_volumes(train_data, path=path, number_idx=2000)
    
train_data_gen = Image3DGenerator(paths["Patient_paths"][:1600], paths["Patient_category"][:1600], batch_size=4)
valid_data_gen = Image3DGenerator(paths["Patient_paths"][1600:], paths["Patient_category"][1600:], batch_size=4)

In [6]:
input_shape = (128, 128, 64, 1)
model = build_3d_network(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 64, 1)  0         
                             ]                                   
                                                                 
 conv3d (Conv3D)             (None, 126, 126, 62, 64)  1792      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 63, 63, 31, 64)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 31, 64)   256       
 ormalization)                                                   
                                                                 
 conv3d_1 (Conv3D)           (None, 61, 61, 29, 64)    110656    
                                                             

In [7]:
history = model.fit(train_data_gen, epochs=100, validation_data=valid_data_gen, callbacks=[model_checkpoint_callback])

Epoch 1/100


KeyboardInterrupt: 

In [ ]:
training_plot(['loss', 'acc'], history)